### 處理訓練集

In [51]:
from keras.datasets import cifar100
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler

In [3]:
# load cifar-100 datasets
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# normalization
scaler_1 = MinMaxScaler()
scaler_1.fit(x_train.reshape(-1,1))
x_train = scaler_1.transform(x_train.reshape(-1,1)).reshape(50000,32,32,3)
scaler_2 = MinMaxScaler()
scaler_2.fit(x_test.reshape(-1,1))
x_test = scaler_2.transform(x_test.reshape(-1,1)).reshape(10000,32,32,3)

# one-hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [4]:
(x_train.shape, y_train.shape), (x_test.shape, y_test.shape)

(((50000, 32, 32, 3), (50000, 100)), ((10000, 32, 32, 3), (10000, 100)))

In [6]:
# split training and validation data with 9:1 (45000 : 5000)
x_t = x_train[:int(x_train.shape[0]*(9/10))]
y_t = y_train[:int(x_train.shape[0]*(9/10))]
x_v = x_train[int(x_train.shape[0]*(9/10)):]
y_v = y_train[int(x_train.shape[0]*(9/10)):]

In [7]:
(x_t.shape,y_t.shape),(x_v.shape,y_v.shape)

(((45000, 32, 32, 3), (45000, 100)), ((5000, 32, 32, 3), (5000, 100)))

### 以 VGG-16 和 inception 的想法做架構 :
#### ● 16 層有參數的層 : 13 層 convolution，3 層 dense。
#### → 13層有參數的層 : 10層 convolution，其中 3 層用到了 inception 的想法，3 層 Dense。
#### ● 6 層沒有參數的層 : 5 層 maxpooling 和 1 層 flatten。 
#### → 9 層沒有參數的層 : 5 層 maxpooling 、 1 層 flatten 和 3 層 Concatenate。

#### 此架構特點為 :
#### 1. 參數量少。

In [48]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from keras.optimizers.schedules import ExponentialDecay
from keras.optimizers import SGD
import tensorflow as tf

In [32]:
# 輸入層
inputs = Input(shape=(32, 32, 3))

# 卷積層和最大池化層
conv1 = Conv2D(8, (3,3), padding='same', activation='relu')(inputs)
conv2 = Conv2D(8, (3,3), padding='same', activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=2)(conv2)
conv3 = Conv2D(64, (3,3), padding='same', activation='relu')(pool1)
conv4 = Conv2D(64, (3,3), padding='same', activation='relu')(conv3)
pool2 = MaxPooling2D(pool_size=2)(conv4)
conv5 = Conv2D(128, (3,3), padding='same', activation='relu')(pool2)

conv6_1 = Conv2D(32, (3,3), padding='same', activation='relu')(conv5)
conv6_2 = Conv2D(32, (3,3), padding='same', activation='relu')(conv5)
conv6_3 = Conv2D(32, (3,3), padding='same', activation='relu')(conv5)
conv6_4 = Conv2D(32, (1,1), padding='same', activation='relu')(conv5)
conv6_5 = Conv2D(32, (3,3), padding='same', activation='relu')(conv6_4)
conv7 = Concatenate(axis=3)([conv6_1,conv6_2,conv6_3,conv6_5])

pool3 = MaxPooling2D(pool_size=2)(conv7)
flat_ = Flatten()(pool3)
# 全聯接層
fc1_ = Dense(2048, activation='relu')(flat_)
fc2_ = Dense(2048, activation='relu')(fc1_)
# 輸出層
outputs_ = Dense(100, activation='softmax')(fc2_)

conv8 = Conv2D(256, (3,3), padding='same', activation='relu')(pool3)

conv9_1 = Conv2D(32, (3,3), padding='same', activation='relu')(conv8)
conv9_2 = Conv2D(32, (3,3), padding='same', activation='relu')(conv8)
conv9_3 = Conv2D(32, (3,3), padding='same', activation='relu')(conv8)
conv9_4 = Conv2D(32, (1,1), padding='same', activation='relu')(conv8)
conv9_5 = Conv2D(32, (3,3), padding='same', activation='relu')(conv9_4)
conv10 = Concatenate(axis=3)([conv9_1,conv9_2,conv9_3,conv9_5])

pool4 = MaxPooling2D(pool_size=2)(conv10)
conv11 = Conv2D(256, (3,3), padding='same', activation='relu')(pool4)

conv12_1 = Conv2D(32, (3,3), padding='same', activation='relu')(conv11)
conv12_2 = Conv2D(32, (3,3), padding='same', activation='relu')(conv11)
conv12_3 = Conv2D(32, (3,3), padding='same', activation='relu')(conv11)
conv12_4 = Conv2D(32, (1,1), padding='same', activation='relu')(conv11)
conv12_5 = Conv2D(32, (3,3), padding='same', activation='relu')(conv12_4)
conv13 = Concatenate(axis=3)([conv12_1,conv12_2,conv12_3,conv12_5])

pool5 = MaxPooling2D(pool_size=2)(conv13)
# 扁平層
flat = Flatten()(pool5)
# 全聯接層
fc1 = Dense(2048, activation='relu')(flat)
fc2 = Dense(2048, activation='relu')(fc1)
# 輸出層
outputs = Dense(100, activation='softmax')(fc2)

# outputss = Dense(100,activation='',outputs+outputs_)

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(Final1, to_file='Final.png',show_shapes=True)
# plot_model(Final2, to_file='Final.png',show_shapes=True)

In [33]:
Final1 = Model(inputs=inputs, outputs=[outputs])
Final2 = Model(inputs=inputs, outputs=[outputs_])
# Final3 = Model(inputs=inputs, outputs=[])

In [34]:
Final1.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, 32, 32, 8)    224         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 32, 32, 8)    584         conv2d_88[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_20 (MaxPooling2D) (None, 16, 16, 8)    0           conv2d_89[0][0]                  
____________________________________________________________________________________________

In [35]:
Final2.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_88 (Conv2D)              (None, 32, 32, 8)    224         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_89 (Conv2D)              (None, 32, 32, 8)    584         conv2d_88[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_20 (MaxPooling2D) (None, 16, 16, 8)    0           conv2d_89[0][0]                  
___________________________________________________________________________________________

In [77]:
# lr_schedule = ExponentialDecay(  
#     initial_learning_rate=1e-2*5,
#     decay_steps=1000,
#     decay_rate=0.8)  # initial_learning_rate * decay_rate ^ (step / decay_steps)

optimizer = SGD(learning_rate=0.05)

Final1.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [74]:
tf.config.experimental_run_functions_eagerly(True)

In [81]:
epoch = 50

hostory = Final1.fit(x_t,y_t,
                    epochs=epoch,
                    batch_size = 64,
                    shuffle=True,
                    validation_data=(x_v, y_v),
                    verbose=1)

Epoch 1/50
704/704 [==============================] - 120s 171ms/step - loss: 4.6056 - categorical_accuracy: 0.0105 - val_loss: 4.6078 - val_categorical_accuracy: 0.0101
Epoch 2/50
704/704 [==============================] - 140s 199ms/step - loss: 4.6056 - categorical_accuracy: 0.0098 - val_loss: 4.6078 - val_categorical_accuracy: 0.0096
Epoch 3/50
704/704 [==============================] - 94s 134ms/step - loss: 4.6056 - categorical_accuracy: 0.0097 - val_loss: 4.6078 - val_categorical_accuracy: 0.0097
Epoch 4/50
704/704 [==============================] - 86s 123ms/step - loss: 4.6055 - categorical_accuracy: 0.0098 - val_loss: 4.6077 - val_categorical_accuracy: 0.0097
Epoch 5/50
704/704 [==============================] - 89s 126ms/step - loss: 4.6056 - categorical_accuracy: 0.0096 - val_loss: 4.6077 - val_categorical_accuracy: 0.0095
Epoch 6/50
704/704 [==============================] - 87s 123ms/step - loss: 4.6056 - categorical_accuracy: 0.0095 - val_loss: 4.6076 - val_categorical_a

KeyboardInterrupt: 

In [ ]:
Final1.save_weights('Final1_weights.h5')

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss plt.plot(history.history['loss']) plt.plot(history.history['val_loss']) plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Final 使用了
#### ● 13層有參數的層 : 10 層 convolution，3 層 dense。(原 VGG16 的最後三層maxpooling後的連續三層的convolution中的後兩層使用inception的想法結合起來)
#### ● 6層沒有參數的層 : 5 個 maxpooling 和 1 個 flatten。

In [6]:
# 輸入層
inputs = Input(shape=(224, 224, 3))

# 卷積層和最大池化層
conv1 = Conv2D(64, (3,3), padding='same', activation='relu')(inputs)
conv2 = Conv2D(64, (3,3), padding='same', activation='relu')(conv1)
pool1 = MaxPooling2D(pool_size=2)(conv2)
conv3 = Conv2D(128, (3,3), padding='same', activation='relu')(pool1)
conv4 = Conv2D(128, (3,3), padding='same', activation='relu')(conv3)
pool2 = MaxPooling2D(pool_size=2)(conv4)

conv5 = Conv2D(256, (3,3), padding='same', activation='relu')(pool2)
conv6 = Conv2D(256, (3,3), padding='same', activation='relu')(conv5)
conv7 = Conv2D(256, (3,3), padding='same', activation='relu')(conv6)
pool3 = MaxPooling2D(pool_size=2)(conv7)
conv8 = Conv2D(512, (3,3), padding='same', activation='relu')(pool3)
conv9 = Conv2D(512, (3,3), padding='same', activation='relu')(conv8)
conv10 = Conv2D(512, (3,3), padding='same', activation='relu')(conv9)
pool4 = MaxPooling2D(pool_size=2)(conv10)
conv11 = Conv2D(512, (3,3), padding='same', activation='relu')(pool4)
conv12 = Conv2D(512, (3,3), padding='same', activation='relu')(conv11)
conv13 = Conv2D(512, (3,3), padding='same', activation='relu')(conv12)
pool5 = MaxPooling2D(pool_size=2)(conv13)
# 扁平層
flat = Flatten()(pool5)
# 全聯接層
fc1 = Dense(4096, activation='relu')(flat)
fc2 = Dense(4096, activation='relu')(fc1)
# 輸出層
outputs = Dense(1000, activation='softmax')(fc2)

In [8]:
like_VGG16_model = Model(inputs=inputs, outputs=outputs)

In [9]:
like_VGG16_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0     